<a href="https://colab.research.google.com/github/JMcKercher/WF2021-SI-Burn-Sev/blob/main/WF_Burn_Sev_SI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculate Burn Severity of the Wooroloo Bushfire 2021

This notebook is going to calculate the burn severity of the Wooroloo Bushfire 2021 using several spectral indices. First you will create the spectral indices and then you will run a Random Forest classifier to classify the burn sevrity of the fire to the highest accuracy.

Options to export the indices as tiffs will be provided at the end of the book.

The base code of this work is sourced from dixondan https://github.com/dixondan/njf-fire-sev/tree/main who I thank greatly for he asistance in getting this to work.

### Set up

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=7xQN3GbBkWxDUgHtqgr-US4y1XhJs8kkfLiqOlcSBRk&tc=E15I9IS-phQGYjUqK49NlGnvzZEHl-DK6cNW42Uy3Vg&cc=SS58S6wwQUBA1nPMwSuZQF8jEe2tsKbu8_O9_SNFZwQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkz8TINqvVr6UsQiHASNEO4oitqOKFYdn0aqcXN5fsP3Hma1oX4xaA

Successfully saved authorization token.


In [ ]:
pip install geopandas

In [ ]:
pip install geemap

In [ ]:
pip install geemap[extra]

In [ ]:
pip install geemap[all]

In [6]:
import pandas as pd
import geopandas as gpd
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import subprocess
import sys
sys.path.insert(0, 'utils')

Upload the required .py files: info_landsat, rf_params, si_calc

In [23]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving rf_params.py to rf_params.py
Saving si_calc.py to si_calc.py
User uploaded file "rf_params.py" with length 1257 bytes
User uploaded file "si_calc.py" with length 941 bytes


In [24]:
import info_landsat as lsat
import si_calc as si
import rf_params as params
import geemap

### Create the Spectral Indices

To calculate the severity of any fire we need the date of fire (start and end date). WHen selecting the fire dates, you need to make sure no burning occurred outside of the slected dates. You will also need to specify the geometry coordinates as x,y for the area that the fire occured.

In [25]:
# enter the coordinates of the fire perimeter and convert to an aoi feature for EE to read
WFg = ee.Geometry.Rectangle([116.008687, -31.694399, 116.358957, -31.815356])
fire_poly = ee.Feature(WFg)

# Date information
date_start = ee.Date('2021-01-28')
date_end = ee.Date('2021-02-15')

In [26]:
# get Landsat data for predicting fires
oliCol = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
etmCol = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
tmCol = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')

#WF Fire filter for Landsat data for aoi and QA
colFilter = ee.Filter.And(
            ee.Filter.bounds(WFg),
            ee.Filter.lt('CLOUD_COVER', 50))

oliCol = oliCol.filter(colFilter).map(lsat.prepOli)
etmCol = etmCol.filter(colFilter).map(lsat.prepEtm)
tmCol = tmCol.filter(colFilter).map(lsat.prepEtm)

landsat = oliCol.merge(etmCol).merge(tmCol)

landsat = landsat.map(lsat.rescale)\
                     .map(si.getNBR)\
                     .map(si.getCSI)\
                     .map(si.getRdNBR)\
                     .map(si.getRBR)

In [27]:
''' define the pre and post collection periods for composites to use for calculating SI '''
fire_poly_buffer = fire_poly.buffer(2000)

# get pre landsat dates
date_pre = date_start.advance(-128, 'day')
date_pre64 = date_start.advance(-64, 'day')
# search window used post-fire
date_post = date_end.advance(64, 'day')
seasonal_date_post = date_end.advance(128, 'day') #Extended window

# filter Landsat data for fire date and extent
ls_temp = landsat.filter(ee.Filter.date(date_pre, date_post))\
                 .filterBounds(fire_poly_buffer.geometry())\
                 .select(['NBR', 'CSI', 'RdNBR', 'RBR'])
# Extended Seasonal
ls_temp = landsat.filter(ee.Filter.date(date_pre, seasonal_date_post))\
                 .filterBounds(fire_poly_buffer.geometry())\
                 .select(['NBR', 'CSI', 'RdNBR', 'RBR'])
# 64 Pre and Post
ls_temp = landsat.filter(ee.Filter.date(date_pre64, date_post))\
                 .filterBounds(fire_poly_buffer.geometry())\
                 .select(['NBR', 'CSI', 'RdNBR', 'RBR'])

#Define Pre and Post Image collection
# Median
pre = ls_temp.filterDate(date_pre, date_start.advance(-1, 'day')).median()
post = ls_temp.filterDate(date_end.advance(1, 'day'), date_post).median() # Original 64-Days post
seasonalpost = ls_temp.filterDate(date_end.advance(1, 'day'), seasonal_date_post).median() # 128-Day post
# Maximum Seasonal
maxpre = ls_temp.filterDate(date_pre, date_start.advance(-1, 'day')).max()
maxpost = ls_temp.filterDate(date_end.advance(1, 'day'), seasonal_date_post).max()
# Minimum Seasonal
minpre = ls_temp.filterDate(date_pre, date_start.advance(-1, 'day')).min()
minpost = ls_temp.filterDate(date_end.advance(1, 'day'), seasonal_date_post).min()
# Minimum 64 Day Pre & Post
minpre64 = ls_temp.filterDate(date_pre64, date_start.advance(-1, 'day')).min()
minpost64 = ls_temp.filterDate(date_end.advance(1, 'day'), date_post).min() # Minimum 64 day post, different to SP


# create a variable for the ee image collection with predicted fire severity data
imcol = ee.ImageCollection('projects/euc-fire-sw-2022/assets/severity_pred_forested')
# this imcol dataset has already been processed to remove non-forestest areas according to Hansen global forest cover

### Calulate the Spectral Indices

In [28]:
from numpy.ma.core import sqrt
''' start working with pre and post image composites to generate predictors'''
# Median (64 Day Post)
# NBR
NBR_pre = pre.select('NBR').rename('NBR_pre')
dNBR_A1 = NBR_pre.subtract(post.select('NBR')).rename('dNBR_A1')
# RdNBR
#NBR_A2 = pre.select('NBR').rename('NBR_A2pre')
#dNBR_A2 = NBR_A2pre.subtract(post.select('NBR')).rename('dNBR_A2')
RdNBR_A1pre = pre.select('RdNBR').rename('RdNBR_A1pre')
RdNBR_A1 = dNBR_A1.divide(RdNBR_A1pre).rename('RdNBR_A1')
# RBR
RBR_A1pre = pre.select('RBR').rename('RBR_A1pre')
RBR_A1 = dNBR_A1.divide(RBR_A1pre).rename('RBR_A1')
# CSI
CSI_A1 = post.select('CSI').rename('CSI_A1')

# Median Seasonal (128 Day Post)
# NBR
NBR_B1 = pre.select('NBR').rename('NBR_B1pre')
dNBR_B1 = NBR_B1.subtract(seasonalpost.select('NBR')).rename('dNBR_B1')
# RdNBR
RdNBR_B1pre = pre.select('RdNBR').rename('RdNBR_B1pre')
RdNBR_B1 = dNBR_B1.divide(RdNBR_B1pre).rename('RdNBR_B1')
# RBR
RBR_B1pre = pre.select('RBR').rename('RBR_B1pre')
RBR_B1 = dNBR_B1.divide(RBR_B1pre).rename('RBR_B1')
# CSI
CSI_B1 = seasonalpost.select('CSI').rename('CSI_B1')

# Maximum Seasonal (128 Day post)
# NBR
NBR_C1 = maxpre.select('NBR').rename('NBR_C1')
dNBR_C1 = NBR_C1.subtract(maxpost.select('NBR')).rename('dNBR_C1')
# RdNBR
RdNBR_C1pre = maxpre.select('RdNBR').rename('RdNBR_C1pre')
RdNBR_C1 = dNBR_C1.divide(RdNBR_C1pre).rename('RdNBR_C1')
# RBR
RBR_C1pre = maxpre.select('RBR').rename('RBR_C1pre')
RBR_C1 = dNBR_C1.divide(RBR_C1pre).rename('RBR_C1')
# CSI
CSI_C1 = maxpost.select('CSI').rename('CSI_C1')

# Minimum Seasonal (128 Day Post)
# NBR
NBR_D1 = minpre.select('NBR').rename('NBR_D1')
dNBR_D1 = NBR_D1.subtract(minpost.select('NBR')).rename('dNBR_D1')
# RdNBR
RdNBR_D1pre = minpre.select('RdNBR').rename('RdNBR_D1pre')
RdNBR_D1 = dNBR_D1.divide(RdNBR_D1pre).rename('RdNBR_D1')
# RBR
RBR_D1pre = minpre.select('RBR').rename('RBR_D1pre')
RBR_D1 = dNBR_D1.divide(RBR_D1pre).rename('RBR_D1')
# CSI
CSI_D1 = minpost.select('CSI').rename('CSI_D1')

# Adjusted Indices based on results of RF model
# RBR 128 day Max Pre 128 day Min Post RBR
RBR_E1pre = maxpre.select('RBR').rename('RBR_E1pre')
dNBR_E1 = NBR_D1.subtract(minpost.select('NBR')).rename('dNBR_E1')
RBR_E1  = dNBR_E1.divide(RBR_E1pre).rename('RBR_E1')
# dNBR Min 64 Day Post
NBR_F1 = minpre64.select('NBR').rename('NBR_F1')
dNBR_F1 = NBR_F1.subtract(minpost64.select('NBR')).rename('dNBR_F1')
# RBR Min 64 Day Post
RBR_F1pre = minpre64.select('RBR').rename('RBR_F1pre')
RBR_F1 = dNBR_F1.divide(RBR_F1pre).rename('RBR_F1')

# List of current variables for VARS to use. IF REQUIRED BY PHENOLOGY OFFSET!
VARS = NBR_pre.addBands([dNBR_A1, CSI_A1, RdNBR_A1, RBR_A1, dNBR_B1, CSI_B1,
                         RdNBR_B1, RBR_B1, dNBR_C1, CSI_C1, RdNBR_C1, RBR_C1,
                         dNBR_D1, CSI_D1, RdNBR_D1, RBR_D1, RBR_E1, dNBR_F1, RBR_F1])

### Visualise the Spectral Indices

Must create visualistion parameters for for the index to use so we can classify different levels of burn severity. This is done by stating the image visualisation parameters. These parameters will not be carried over when the image is exported as a TIFF, but is required to display the image collection. The final image must be converted into a mosaic before it can be visualised on the map.

In [29]:
# Mosaic the visualization layers and display (or export).
# Original data
A1_CSI = ee.ImageCollection([CSI_A1]).mosaic()
A1_RdNBR = ee.ImageCollection([RdNBR_A1]).mosaic()
A1_dNBR = ee.ImageCollection([dNBR_A1]).mosaic()
A1_RBR = ee.ImageCollection([RBR_A1]).mosaic()
# Seasonal Post
B1_CSI = ee.ImageCollection([CSI_B1]).mosaic()
B1_RdNBR = ee.ImageCollection([RdNBR_B1]).mosaic()
B1_dNBR = ee.ImageCollection([dNBR_B1]).mosaic()
B1_RBR = ee.ImageCollection([RBR_B1]).mosaic()
# Maximum Seasonal
C1_CSI = ee.ImageCollection([CSI_C1]).mosaic()
C1_RdNBR = ee.ImageCollection([RdNBR_C1]).mosaic()
C1_dNBR = ee.ImageCollection([dNBR_C1]).mosaic()
C1_RBR = ee.ImageCollection([RBR_C1]).mosaic()
# Minimum Seasonal
D1_CSI = ee.ImageCollection([CSI_D1]).mosaic()
D1_RdNBR = ee.ImageCollection([RdNBR_D1]).mosaic()
D1_dNBR = ee.ImageCollection([dNBR_D1]).mosaic()
D1_RBR = ee.ImageCollection([RBR_D1]).mosaic()
# Adjusted Indicies
E1_RBR = ee.ImageCollection([RBR_E1]).mosaic()
F1_dNBR = ee.ImageCollection([dNBR_F1]).mosaic()
F1_RBR = ee.ImageCollection([RBR_F1]).mosaic()

#Set Visualisation Parameters
CSIparams = {'min': 0, 'max': 1, 'palette': ['#FFFFFF', '#696969', '#000000']}
RdNBRparams = {'min': 0, 'max': 2, 'palette': ['#4682B4', '#FAFAD2', '#FF5400','#C71585']}
dNBRparams = {'min': 0, 'max': 0.5, 'palette': ['#4682B4', '#FAFAD2', '#FF5400','#C71585']}
RBRparams = {'min': 0, 'max': 2, 'palette': ['#4682B4', '#FAFAD2', '#FF5400','#C71585']}
RBRModparams = {'min': 0, 'max': 1, 'palette': ['#4682B4', '#FAFAD2', '#FF5400','#C71585']}

#Clip the fire boundary you want to catergorize
#CSI
CSI_A1F = A1_CSI.clip(WFg)
CSI_B1F = B1_CSI.clip(WFg)
CSI_C1F = C1_CSI.clip(WFg)
CSI_D1F = D1_CSI.clip(WFg)
#RdNBR
RdNBR_A1F = A1_RdNBR.clip(WFg)
RdNBR_B1F = B1_RdNBR.clip(WFg)
RdNBR_C1F = C1_RdNBR.clip(WFg)
RdNBR_D1F = D1_RdNBR.clip(WFg)
#dNBR
dNBR_A1F = A1_dNBR.clip(WFg)
dNBR_B1F = B1_dNBR.clip(WFg)
dNBR_C1F = C1_dNBR.clip(WFg)
dNBR_D1F = D1_dNBR.clip(WFg)
#RBR
RBR_A1F = A1_RBR.clip(WFg)
RBR_B1F = B1_RBR.clip(WFg)
RBR_C1F = C1_RBR.clip(WFg)
RBR_D1F = D1_RBR.clip(WFg)
# Adjusted
RBR_E1F = E1_RBR.clip(WFg)
dNBR_F1F = F1_dNBR.clip(WFg)
RBR_F1F = F1_RBR.clip(WFg)

In [47]:
# Map the SI
x,y = fire_poly.centroid().getInfo()['geometry']['coordinates']
# call geemap for plotting
Map = geemap.Map(center=(y,x), zoom = 12)
Map.add_basemap('HYBRID')

# add image boundary of interest
#Map.addLayer(fire_poly)

'''To view the indices of your choice, simply comment the indicies you don't want to display, or view them all at once'''
# CSI
'''Map.addLayer(CSI_A1F, CSIparams, 'CSI_A1')
Map.addLayer(CSI_B1F, CSIparams, 'CSI_B1')
Map.addLayer(CSI_C1F, CSIparams, 'CSI_C1')
Map.addLayer(CSI_D1F, CSIparams, 'CSI_D1')
#RdNBR
Map.addLayer(RdNBR_A1F, RdNBRparams, 'RdNBR_A1')
Map.addLayer(RdNBR_B1F, RdNBRparams, 'RdNBR_B1')
Map.addLayer(RdNBR_C1F, RdNBRparams, 'RdNBR_C1')
Map.addLayer(RdNBR_D1F, RdNBRparams, 'RdNBR_D1')
#dNBR
Map.addLayer(dNBR_A1F, dNBRparams, 'dNBR_A1')
Map.addLayer(dNBR_B1F, dNBRparams, 'dNBR_B1')
Map.addLayer(dNBR_C1F, dNBRparams, 'dNBR_C1')
Map.addLayer(dNBR_D1F, dNBRparams, 'dNBR_D1')
#RBR
Map.addLayer(RBR_A1F, RBRparams, 'RBR_A1')
Map.addLayer(RBR_B1F, RBRparams, 'RBR_B1')
Map.addLayer(RBR_C1F, RBRparams, 'RBR_C1')
Map.addLayer(RBR_D1F, RBRparams, 'RBR_D1')'''
# Adjusted
Map.addLayer(RBR_E1F, RBRparams, 'RBR_E1')
Map.addLayer(dNBR_F1F, dNBRparams, 'dNBR_F1')
Map.addLayer(RBR_F1F, RBRModparams, 'RBR_F1')

# Create a legend for maps to use
legend_dict = {
    'Unburnt': '#4682B4',
    'Low': '#FAFAD2',
     'Moderate': '#FF5400',
    'High': '#C71585'}
Map.add_legend(title="Burn Severity", legend_dict=legend, position='bottomright')

Map

Map(center=[-31.7549841114179, 116.18382199999944], controls=(WidgetControl(options=['position', 'transparent_…

### Random Forest Classifier

In [48]:
#Create a combined variable of all indices
AllIndex = ['CSI_A1', 'CSI_B1', 'CSI_D1', 'CSI_E1', 'dNBR_A1','dNBR_B1', 'dNBR_C1', 'dNBR_E1', 'RdNBR_A1','RdNBR_B1',
            'RdNBR_C1', 'RdNBR_D1', 'RBR_A1','RBR_B1', 'RBR_C1', 'RBR_D1', 'RBR_E1', 'dNBR_F1', 'RBR_F1']
RF_F1dNBR = ['dNBR_F1'] #Transforms the image to a listed image
RF_F1RBR = ['RBR_F1'] #Transforms the image to a listed image

# Upload the table and convert it to a feature collection so EE can read it.
'''This function calls the CSV file from google earth engine directly, the file is uploaded into the assests area of the program and called by the below code'''
# Contains results from all indices
RFCdat = ee.FeatureCollection('projects/ee-jmckercher/assets/GC_RFC') # CSV from GEE
'''Note: Class variables must be numeric and start from 0: 0 = Unburnt, 1 = Low, 2 = Moderate, 3 = High'''
# Pull training and test data from all indices
train = RFCdat.filter(ee.Filter.stringContains("train_test", 'Train'))
test = RFCdat.filter(ee.Filter.stringContains("train_test", 'Test'))

# Contains results of just the dNBR min 64 Days
dNBRF1_rf = ee.FeatureCollection('projects/ee-jmckercher/assets/GC_RFC_dNBR')
# Call EE to filter train data points and test data points to divide the data set into test and train sets.
F1_train = dNBRF1_rf.filter(ee.Filter.stringContains("train_test", 'Train')) #Specifically for dNBR Min 64
F1_test = dNBRF1_rf.filter(ee.Filter.stringContains("train_test", 'Test')) #Specifically for dNBR Min 64

# Contains results of just the RBR min 64 Days with the outliers removed
RBRF1_rf = ee.FeatureCollection('projects/ee-jmckercher/assets/RFC_RBR_ORem')
# Call EE to filter train data points and test data points to divide the data set into test and train sets.
RBRF1_train = RBRF1_rf.filter(ee.Filter.stringContains("train_test", 'Train')) #Specifically for dNBR Min 64
RBRF1_test = RBRF1_rf.filter(ee.Filter.stringContains("train_test", 'Test')) #Specifically for dNBR Min 64

# Random Forest Classifier Layout
'''classifier = ee.Classifier.smileRandomForest(500, seed=123).train(
     features= train,
     classProperty= 'SA_class',
     inputProperties= AllIndex)'''

# dNBR min Classifier
F1_classifier = ee.Classifier.smileRandomForest(500, seed=123).train(
     features= F1_train,
     classProperty= 'SA_class',
     inputProperties= RF_F1dNBR) #Increasing amount of trees to 200 increases accuracy.

# dNBR min Classifier
RBRF1_classifier = ee.Classifier.smileRandomForest(500, seed=123).train(
     features= RBRF1_train,
     classProperty= 'SA_class',
     inputProperties= RF_F1RBR) #Increasing amount of trees to 200 increases accuracy.

RBRF1_trainAcc = RBRF1_classifier.confusionMatrix()
print('Results of Classifier','\n',RBRF1_classifier.explain().getInfo())
print('Training Accuracy: ', RBRF1_trainAcc.accuracy().getInfo())
print('Training Kappa: ', RBRF1_trainAcc.kappa().getInfo())

Results of Classifier 
 {'classes': [0, 1, 2, 3], 'importance': {'RBR_F1': 1919.4973420410554}, 'numberOfTrees': 500, 'outOfBagErrorEstimate': 0.40860215053763443, 'trees': ['n= 48\nnode), split, n, loss, yval, (yprob)\n* denotes terminal node\n1) root 48 34.458 3 (0.28846 0.15385 0.19231 0.36538)\n 2) RBR_F1<=0.309286 26 15.615 0 (0.50000 0.26667 0.20000 0.033333)\n  4) RBR_F1<=0.0299334 17 5.1765 0 (0.71429 0.14286 0.095238 0.047619)\n   8) RBR_F1<=-0.0124670 15 3.6000 0 (0.73684 0.10526 0.10526 0.052632)\n    16) RBR_F1<=-0.185090 2 1.0000 0 (0.33333 0.33333 0.16667 0.16667)\n     32) RBR_F1<=-0.216695 1 0.0000 0 (0.40000 0.20000 0.20000 0.20000) *\n     33) RBR_F1>-0.216695 1 0.0000 1 (0.20000 0.40000 0.20000 0.20000) *\n    17) RBR_F1>-0.185090 13 1.8462 0 (0.76471 0.058824 0.11765 0.058824)\n     34) RBR_F1<=-0.0845444 6 1.6667 0 (0.60000 0.10000 0.20000 0.10000)\n      68) RBR_F1<=-0.0967464 5 0.0000 0 (0.66667 0.11111 0.11111 0.11111) *\n      69) RBR_F1>-0.0967464 1 0.0000 2 (

In [49]:
# Classify the validation data.
RBRF1_Validate = RBRF1_test.classify(RBRF1_classifier)
RBRF1_Mat = RBRF1_Validate.errorMatrix('SA_class', 'classification')
RBRF1_Mat_info = RBRF1_Mat.getInfo()
print('Validation Accuracy', RBRF1_Mat.accuracy().getInfo())
print('Validation Kappa', RBRF1_Mat.kappa().getInfo())
print('Specified name and order of the rows and columns', RBRF1_Mat.order().getInfo()) # Returns the name and order of the rows and columns of the matrix.
print('Validation Confusion Matrix', RBRF1_Mat_info)
print('F-Score', RBRF1_Mat.fscore(1).getInfo()) # Computes the Fβ-score for the confusion matrix: The higher an F-score, the more accurate a model is. The lower an F-score, the less accurate a model is.
print("Producer's accuracy", RBRF1_Mat.producersAccuracy().getInfo()) # Computes the producer's accuracy of a confusion matrix defined as (correct / total) for each column, the probability that a value in a given class was classified correctly.
print("Consumer's accuracy", RBRF1_Mat.consumersAccuracy().getInfo()) # Computes the consumer's accuracy (reliability) of a confusion matrix defined as (correct / total) for each row.
'''the probability that a value predicted to be in a certain class really is that class. The probability is based on the fraction of correctly predicted values to the total number of values predicted to be in a class'''

Validation Accuracy 0.6111111111111112
Validation Kappa 0.42465753424657543
Specified name and order of the rows and columns [0, 1, 2, 3]
Validation Confusion Matrix [[3, 0, 2, 0], [3, 3, 1, 0], [0, 1, 2, 4], [0, 0, 3, 14]]
F-Score [0.5454545454545454, 0.5454545454545454, 0.26666666666666666, 0.7999999999999999]
Producer's accuracy [[0.6], [0.42857142857142855], [0.2857142857142857], [0.8235294117647058]]
Consumer's accuracy [[0.5, 0.75, 0.25, 0.7777777777777778]]


'the probability that a value predicted to be in a certain class really is that class. The probability is based on the fraction of correctly predicted values to the total number of values predicted to be in a class'

In [51]:
# Use the classifier to predict on an image
classified = RBR_F1.classify(RBRF1_classifier)
classified = classified.select('classification')
classified = classified.clip(WFg)
classified = classified.toByte() # only has values from 0-3 so reduce file size
classified.getInfo()

{'type': 'Image',
 'bands': [{'id': 'classification',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [2, 2],
   'origin': [115, -33],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'system:footprint': {'type': 'Polygon',
   'coordinates': [[[116.008687, -31.815356],
     [116.358957, -31.815356],
     [116.358957, -31.694399],
     [116.008687, -31.694399],
     [116.008687, -31.815356]]]}}}

In [53]:
# put it on the map
x,y = fire_poly.centroid().getInfo()['geometry']['coordinates']
# call geemap for plotting
Map = geemap.Map(center=(y, x), zoom = 12)
Map.add_basemap('HYBRID')

# Visualize burn area
ClassParams = {'min': 0, 'max': 3, 'palette' : ['#4682B4','#FAFAD2', '#FF5400','#C71585']}

#Add a Legend
legend = {
    'Unburnt': '#4682B4',
    'Low': '#FAFAD2',
     'Moderate': '#FF5400',
    'High': '#C71585'}
Map.add_legend(title="Burn Severity", legend_dict=legend, position='bottomleft')

# Clip the Image that was classified & add to map to determine the pixel value of each classification
#dNBR_F1F = dNBR_F1.clip(WFg)
#Map.addLayer(dNBR_F1F, dNBRparams,'dNBR') # Image that was classified
RBR_F1F = dNBR_F1.clip(WFg)

Map.addLayer(RBR_F1F, RBRparams,'RBR')

Map.addLayer(classified.select('classification'), ClassParams, 'RF Classification') # Classified Image
Map

Map(center=[-31.7549841114179, 116.18382199999944], controls=(WidgetControl(options=['position', 'transparent_…